In [1]:
%pip install httpx

Note: you may need to restart the kernel to use updated packages.


In [2]:
import httpx

from pathlib import Path

# Comparing prod and dev

Ok, not much difference here.

In [3]:
prod = "https://wersdoerfer.de"
staging = "https://homepage.staging.django-cast.com"
dev = "http://localhost:8000"

# For About

In [4]:
%%time
r = httpx.get(f"{prod}/about/")
r.raise_for_status()
content = r.text
print(len(content))

2765
CPU times: user 9.85 ms, sys: 3.19 ms, total: 13 ms
Wall time: 120 ms


In [5]:
%%time
r = httpx.get(f"{staging}/about/")
r.raise_for_status()
content = r.text
print(len(content))

2765
CPU times: user 13.1 ms, sys: 2.26 ms, total: 15.3 ms
Wall time: 113 ms


In [6]:
%%time
r = httpx.get(f"{dev}/about/")
r.raise_for_status()
content = r.text
print(len(content))

12480
CPU times: user 14.5 ms, sys: 2.65 ms, total: 17.1 ms
Wall time: 467 ms


In [7]:
reverse("about")

'/about/'

# For a Blog Index Page

In [8]:
# blog_slug = "ephes_blog"
blog_slug = "das_claas_und_nora_blog"

In [9]:
%%time
r = httpx.get(f"{prod}/blogs/{blog_slug}/", timeout=60)
r.raise_for_status()
content = r.text
print(len(content))

120631
CPU times: user 11.2 ms, sys: 2.02 ms, total: 13.2 ms
Wall time: 6.36 s


In [10]:
with Path("/tmp/production.html").open("w") as f:
    f.write(content)

In [11]:
%%time
r = httpx.get(f"{staging}/blogs/{blog_slug}/", timeout=60)
r.raise_for_status()
content = r.text
print(len(content))

112766
CPU times: user 21.3 ms, sys: 3.42 ms, total: 24.7 ms
Wall time: 285 ms


In [12]:
with Path("/tmp/staging.html").open("w") as f:
    f.write(content)

In [13]:
%%time
r = httpx.get(f"{dev}/blogs/{blog_slug}/")
r.raise_for_status()
content = r.text
print(len(content))

122566
CPU times: user 14.6 ms, sys: 2.74 ms, total: 17.4 ms
Wall time: 431 ms


In [14]:
%%time
r = httpx.head(f"{dev}/blogs/{blog_slug}/")
r.raise_for_status()
content = r.text
print(len(content))

0
CPU times: user 5.62 ms, sys: 1.08 ms, total: 6.7 ms
Wall time: 354 ms


In [15]:
print(f"{dev}/blogs/{blog_slug}/")

http://localhost:8000/blogs/das_claas_und_nora_blog/


# What About the Feed?

In [16]:
%%time
print("Production!")
r = httpx.get(f"{prod}/blogs/{blog_slug}/feed/rss.xml")
r.raise_for_status()
content = r.text
print(len(content))

Production!
5293024
CPU times: user 31.4 ms, sys: 15.1 ms, total: 46.6 ms
Wall time: 3.79 s


In [17]:
%%time
print("Staging!")
r = httpx.get(f"{staging}/blogs/{blog_slug}/feed/rss.xml", timeout=600)
r.raise_for_status()
content = r.text
print(len(content))

Staging!
5293024
CPU times: user 32 ms, sys: 7.73 ms, total: 39.7 ms
Wall time: 3.16 s


In [18]:
%%time
print("Development!")
r = httpx.get(f"{dev}/blogs/{blog_slug}/feed/rss.xml", timeout=60)
r.raise_for_status()
content = r.text
print(len(content))

Development!
5293023
CPU times: user 15.3 ms, sys: 7.33 ms, total: 22.6 ms
Wall time: 7.7 s


In [19]:
print(blog_slug)

das_claas_und_nora_blog


## Claas Blog

In [20]:
blog_slug = "das_claas_und_nora_blog"

In [21]:
%%time
print("Development!")
r = httpx.get(f"{dev}/blogs/{blog_slug}/feed/rss.xml", timeout=60)
r.raise_for_status()
content = r.text
print(len(content))

Development!
5293023
CPU times: user 12.8 ms, sys: 6.49 ms, total: 19.3 ms
Wall time: 40.5 ms


In [22]:
%%time
print("Staging!")
r = httpx.get(f"{staging}/blogs/{blog_slug}/feed/rss.xml", timeout=600)
r.raise_for_status()
content = r.text
print(len(content))

Staging!
5293024
CPU times: user 24.8 ms, sys: 7.91 ms, total: 32.7 ms
Wall time: 197 ms


In [23]:
%%time
print("Production!")
r = httpx.get(f"{prod}/blogs/{blog_slug}/feed/rss.xml", timeout=600)
r.raise_for_status()
content = r.text
print(len(content))

Production!
5293024
CPU times: user 37.5 ms, sys: 9.4 ms, total: 46.9 ms
Wall time: 201 ms


## What about the feed call?

In [24]:
from django.test.client import RequestFactory
from cast.feeds import LatestEntriesFeed

In [25]:
%%time
feed_view = LatestEntriesFeed()
request_factory = RequestFactory()
request = request_factory.get(f"/blogs/{blog_slug}/feed/rss.xml")
r = feed_view(request, slug=blog_slug)

CPU times: user 1.62 s, sys: 89.4 ms, total: 1.71 s
Wall time: 1.76 s


In [26]:
print(len(r.content))

5294223


# How long does the serve call take?

In [27]:
from django.test.client import RequestFactory
from django_htmx.middleware import HtmxDetails

In [28]:
request_factory = RequestFactory()
request = request_factory.get("/")
request.htmx = HtmxDetails(request)

In [29]:
blog = Blog.objects.get(slug=blog_slug)

In [30]:
%%time
response = blog.serve(request).render()
print(len(response.content))

kwargs['post_data'] <cast.cache.PagedPostData object at 0x11ef38a10>
112847
CPU times: user 70.5 ms, sys: 9.44 ms, total: 80 ms
Wall time: 91.3 ms


In [31]:
[b.slug for b in Blog.objects.all()]

['das_claas_und_nora_blog', 'ephes_blog']

In [32]:
#Rendition.objects.all().delete()

In [33]:
slug = "weeknotes-2023-07-24"
post = Post.objects.get(slug=slug)

In [34]:
post.sync_media_ids()

In [35]:
post.media_ids_from_db

{'image': set(), 'video': set(), 'gallery': {228}, 'audio': set()}

In [36]:
post.media_ids_from_body

{'gallery': {228}}

In [37]:
[gallery] = post.galleries.all()

In [38]:
gallery


<Gallery: Gallery object (228)>

In [39]:
gallery.images.all()

<ImageQuerySet [<Image: Marktplatz in Vannes>, <Image: Fachwerkhäuser in Vannes>, <Image: Fensterspiegelung>, <Image: Sonnenstern an der Saline>, <Image: Fernglas zum Beobachten der Vögel an der Saline>, <Image: Salz in der Saline>, <Image: Windrad neben der Saline>, <Image: Ein paar Blumen am Wegesrand>, <Image: Atlantikküste vor Quiberon>, <Image: Steilküste vor Quiberon>, <Image: Strand in Quiberon>, <Image: Am Strand von Mousterian>, <Image: Frau am Strand von Mousterian>]>

In [40]:
post.images.all()

<ImageQuerySet []>

In [41]:
Image.objects.count()

2369

## Count Queries

In [42]:
import sqlparse

from django.db import connection, reset_queries

In [43]:
def count_queries(func):
    reset_queries()
    response = func()
    print(len(response.content))
    return len(connection.queries)


def show_queries(queries):
    for query in queries:
        formatted_sql = sqlparse.format(query['sql'], reindent=True, keyword_case='upper')
        print(formatted_sql)    

In [44]:
feed_view = LatestEntriesFeed()
request_factory = RequestFactory()
request = request_factory.get(f"/blogs/{blog_slug}/feed/rss.xml")
func = lambda: feed_view(request, slug=blog_slug)
count_queries(func)

5294223


305

In [45]:
request_factory = RequestFactory()
request = request_factory.get("/")
htmx


NameError: name 'htmx' is not defined

In [ ]:
blog = Blog.objects.get(slug=blog_slug)

In [ ]:
%%time
num_queries = count_queries(blog.serve(request).render)
print(num_queries)

In [ ]:
# show_queries(connection.queries)